In [22]:
with open('file.txt', 'r') as file:
    lines = file.read().strip().split('\n')


In [ ]:
def parse_input(lines):
    shapes = {}
    regions = []

    i = 0

    while i < len(lines):
        line = lines[i].strip()

        # Check if shape: 
        if line and line.endswith(':'):
            shape_id = int(line[:-1])
            shape_grid = []
            i += 1

            while i < len(lines) and lines[i].strip() and not lines[i].strip().endswith(':') and 'x' not in lines[i]:
                shape_grid.append(lines[i].strip())
                i +=1
            
            shapes[shape_id] = shape_grid

        # Check if region:
        elif 'x' in line:
            parts = line.split(':')
            size_parts = parts[0].strip().split('x')
            width = int(size_parts[0])
            height = int(size_parts[1])
            
            counts = list(map(int, parts[1].strip().split()))
            regions.append((width, height, counts))
            i += 1
        else:
            i += 1
    
    return shapes, regions



In [24]:
# Flip and handle shapes:

def get_shape_coords(shape_grid):
    """Extract (row, col) coordinates of # cells in a shape"""
    coords = []
    for r, row in enumerate(shape_grid):
        for c, cell in enumerate(row):
            if cell == '#':
                coords.append((r, c))
    return coords

def normalize_coords(coords):
    """Normalize coordinates to start at (0, 0)"""
    if not coords:
        return []
    min_r = min(r for r, c in coords)
    min_c = min(c for r, c in coords)
    return [(r - min_r, c - min_c) for r, c in coords]

def get_rotations_and_flips(coords):
    """Generate all rotations and flips of a shape"""
    variations = set()
    
    # Original
    variations.add(tuple(sorted(normalize_coords(coords))))
    
    # Rotate 90, 180, 270
    for _ in range(3):
        coords = [(c, -r) for r, c in coords]
        variations.add(tuple(sorted(normalize_coords(coords))))
    
    # Flip horizontally
    coords = [(r, -c) for r, c in coords]
    variations.add(tuple(sorted(normalize_coords(coords))))
    
    # Rotate flipped version
    for _ in range(3):
        coords = [(c, -r) for r, c in coords]
        variations.add(tuple(sorted(normalize_coords(coords))))
    
    return [list(v) for v in variations]


In [25]:
def can_place_shape(grid, shape_coords, start_r, start_c):
    """Check if shape can be placed at position"""
    for dr, dc in shape_coords:
        r, c = start_r + dr, start_c + dc
        if r < 0 or r >= len(grid) or c < 0 or c >= len(grid[0]):
            return False
        if grid[r][c]:
            return False
    return True

def place_shape(grid, shape_coords, start_r, start_c, mark):
    """Place shape on grid with given mark"""
    for dr, dc in shape_coords:
        r, c = start_r + dr, start_c + dc
        grid[r][c] = mark

def remove_shape(grid, shape_coords, start_r, start_c):
    """Remove shape from grid"""
    for dr, dc in shape_coords:
        r, c = start_r + dr, start_c + dc
        grid[r][c] = None


def solve_region(width, height, shape_list, timeout=5.0):
    """Try to fit all shapes into region using backtracking"""
    import time
    
    # Pre-compute all variations for each unique shape
    shape_variations = {}
    for shape_id, coords in shape_list:
        key = (shape_id, tuple(sorted(coords)))
        if key not in shape_variations:
            shape_variations[key] = get_rotations_and_flips(coords)
    
    grid = [[None for _ in range(width)] for _ in range(height)]
    start_time = time.time()
    
    def backtrack(idx):
        # Timeout check
        if time.time() - start_time > timeout:
            return None  # Timeout
        
        if idx == len(shape_list):
            return True  # All shapes placed successfully
        
        shape_id, shape_coords = shape_list[idx]
        key = (shape_id, tuple(sorted(shape_coords)))
        
        # Try all rotations/flips of this shape
        for variation in shape_variations[key]:
            # Try all positions in grid
            for r in range(height):
                for c in range(width):
                    if can_place_shape(grid, variation, r, c):
                        place_shape(grid, variation, r, c, f"{shape_id}_{idx}")
                        
                        result = backtrack(idx + 1)
                        if result is True:
                            return True
                        elif result is None:
                            remove_shape(grid, variation, r, c)
                            return None  # Timeout
                        
                        remove_shape(grid, variation, r, c)
        
        return False
    
    result = backtrack(0)
    return result is True

In [26]:
def check_region(width, height, counts, shapes, timeout=5.0):
    """Check if a region can fit all required presents"""
    # Build list of shapes to place
    shape_list = []
    for shape_id, count in enumerate(counts):
        if count > 0:
            coords = get_shape_coords(shapes[shape_id])
            for _ in range(count):
                shape_list.append((shape_id, coords))
    
    # Quick area check - count only the # cells in each shape
    total_cells = sum(len(coords) for _, coords in shape_list)
    
    if total_cells > width * height:
        return False
    
    result = solve_region(width, height, shape_list, timeout)
    return result

In [27]:
# === Solve for real data ===

shapes_real, regions_real = parse_input(lines)

print(f"Real data has {len(shapes_real)} shapes and {len(regions_real)} regions\n")

solvable_real = 0

for i, (width, height, counts) in enumerate(regions_real):
    can_fit = check_region(width, height, counts, shapes_real)
    
    if can_fit:
        solvable_real += 1
        

print(f"\n{'='*60}")
print("PART 1 ANSWER")
print(f"{'='*60}")
print(f"Regions that can fit all presents: {solvable_real}")
print(f"{'='*60}")

Real data has 6 shapes and 1000 regions


PART 1 ANSWER
Regions that can fit all presents: 505
